In [4]:
import numpy as np
import pandas as pd
sms= pd.read_csv('14.2SMS.tsv', header= None, delimiter='\t', names=['Target', 'Message'])
sms.head()

,Target,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
sms['Target']=sms['Target'].map({'ham':0, 'spam':1})
sms.head()

,Target,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
import re
def clean(x):
    x = re.sub(r'<.*>', '', x)  
    x = re.sub(r'[^a-zA-Z0-9]',' ',x)
    x = re.sub(r'\s+',' ',x)          
    return x.lower()

In [7]:
sms['Message']= sms['Message'].apply(clean)
sms.head()

,Target,Message
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in 2 a wkly comp to win fa cup fina...
3,0,u dun say so early hor u c already then say
4,0,nah i don t think he goes to usf he lives arou...


In [8]:
sms.shape

(5572, 2)

In [9]:
sms.isnull().sum()

Target     0
Message    0
dtype: int64

In [10]:
x= sms.Message
y= sms.iloc[:, 0]

In [11]:
x.shape

(5572,)

In [12]:
y.shape

(5572,)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= .20, random_state=0, shuffle= True)

In [14]:
np.bincount(y_test)

array([955, 160], dtype=int64)

In [15]:
x_train.shape

(4457,)

In [16]:
y_train.shape

(4457,)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer= CountVectorizer(stop_words= 'english', max_df= .5, min_df= 10)

In [18]:
x_train= vectorizer.fit_transform(x_train).toarray()
x_test= vectorizer.transform(x_test).toarray()

In [19]:
x_train.shape

(4457, 697)

In [20]:
from sklearn.metrics import confusion_matrix,recall_score

In [21]:
from sklearn.linear_model import LogisticRegression
log= LogisticRegression(C=1,solver='liblinear', n_jobs= 1, multi_class='ovr')
log.fit(x_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [22]:
y_log_pred= log.predict(x_test)

In [23]:
recall_score(y_test, y_log_pred)

0.875

In [24]:
confusion_matrix(y_test, y_log_pred)

array([[952,   3],
       [ 20, 140]], dtype=int64)

In [25]:
from sklearn.naive_bayes import MultinomialNB
mnb= MultinomialNB()
mnb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
y_mnb_pred= mnb.predict(x_test)

In [27]:
recall_score(y_test, y_mnb_pred)

0.9375

In [28]:
confusion_matrix(y_test, y_mnb_pred)

array([[945,  10],
       [ 10, 150]], dtype=int64)

In [29]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(n_estimators=50, max_features='auto', n_jobs=1)
rf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [30]:
y_rf_pred= rf.predict(x_test)

In [31]:
recall_score(y_test, y_rf_pred)

0.9

In [32]:
confusion_matrix(y_test, y_rf_pred)

array([[953,   2],
       [ 16, 144]], dtype=int64)